In [ ]:
import glob
import re
import extract_textbooks
import json
import matplotlib.pyplot as plt

In [ ]:
# Parse the books into individual .txt files
extract_textbooks.process_textbooks_multiprocess('astro_textbooks/')
extract_textbooks.process_textbooks_multiprocess('physics_textbooks/')

In [ ]:
# Preprocess, to get a list of books, where each book is a list of paragraphs
book_paths = glob.glob('textbooks_extracted/*.txt')
books_paragraphs = [extract_textbooks.preprocess_text(file_path) for file_path in book_paths]

In [ ]:
# Mark each paragraph as good or bad based on whether the rate of certain
# characters is within the distribution
%matplotlib widget
extract_textbooks.histogram_percentages(books_paragraphs)

In [ ]:
bounds = {
    'spaces': (6, 24),
    'digits': (0, 15),
    'capital_letters': (1, 23),
    'lowercase_letters': (50, 95),
    'newlines': (0, 5),
    'backslashes': (0, 5),
    'periods': (0, 8),
    'exclamation_marks': (0, 5),
    'question_marks': (0, 6)
}

# Filter the textbooks
filtered_books_paragraphs = extract_textbooks.filter_textbooks(books_paragraphs, bounds)

In [ ]:
# save the dataset in JSONL
with open('textbooks.jsonl', 'w', encoding='utf-8') as file:
    for book in filtered_books_paragraphs:
        for para in book:
            file.write(json.dumps({'text': para}) + '\n')
